In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment',None)
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pickle import dump
from sklearn.preprocessing import MinMaxScaler
from pandas_profiling import ProfileReport
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

C:\Users\wm_herrera\Anaconda3\envs\tensorflow_keras\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\wm_herrera\Anaconda3\envs\tensorflow_keras\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import codigo
from codigo import utils
from codigo import model_utils

# Carga del conjunto de datos inicial

In [3]:
df_Inicial = pd.read_csv(r'dataset/FreeStyle.csv')

# Vista previa del conjunto de datos

In [4]:
df_Inicial.head()

,ID,Date,Glucose level
0,LIB193263,2020-06-09 19:08:00,99.0
1,LIB193263,2020-06-09 19:23:00,92.0
2,LIB193263,2020-06-09 19:38:00,86.0
3,LIB193263,2020-06-09 19:53:00,85.0
4,LIB193263,2020-06-09 20:08:00,85.0


,ID,Date,Glucose level
0,LIB193263,2020-06-09 19:08:00,99.0
1,LIB193263,2020-06-09 19:23:00,92.0
2,LIB193263,2020-06-09 19:38:00,86.0
3,LIB193263,2020-06-09 19:53:00,85.0
4,LIB193263,2020-06-09 20:08:00,85.0


In [5]:
df_Inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000001 entries, 0 to 3000000
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ID             object 
 1   Date           object 
 2   Glucose level  float64
dtypes: float64(1), object(2)
memory usage: 68.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000001 entries, 0 to 3000000
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ID             object 
 1   Date           object 
 2   Glucose level  float64
dtypes: float64(1), object(2)
memory usage: 68.7+ MB


# Ajuste de tipos de datos

In [6]:
df_Inicial['Date']=pd.to_datetime(df_Inicial['Date'])

In [7]:
df_Inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000001 entries, 0 to 3000000
Data columns (total 3 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ID             object        
 1   Date           datetime64[ns]
 2   Glucose level  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 68.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000001 entries, 0 to 3000000
Data columns (total 3 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ID             object        
 1   Date           datetime64[ns]
 2   Glucose level  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 68.7+ MB


In [8]:
df_Inicial['Date'].min()

Timestamp('2016-10-05 08:47:00')

Timestamp('2016-10-05 08:47:00')

In [9]:
df_Inicial['Date'].max()

Timestamp('2044-03-22 03:44:00')

Timestamp('2044-03-22 03:44:00')

# Limpieza del conjunto de datos

## Elimininamos los registros duplicados por paciente y fecha

In [ ]:
df_Inicial=df_Inicial.drop_duplicates(['ID','Date'],keep='first')
df_Inicial.reset_index(drop=True, inplace=True)

In [ ]:
df_Inicial.info()

## Seleccionamos el rango de fechas validas

In [ ]:
DateStart='2018-01-06';
df_depurado=df_Inicial[df_Inicial['Date'] >=DateStart]
df_depurado.info()

In [ ]:
DateEnd='2022-03-21';
df_depurado=df_depurado[df_depurado['Date'] <=DateEnd]
df_depurado.info()

In [ ]:
df_depurado.describe()

In [ ]:
df=df_depurado

In [ ]:
df['Date'].min()

In [ ]:
df['Date'].max()

In [ ]:
df.to_csv('dataset/generado/datasetDepurado.csv', index=False) 

In [ ]:
df_Inicial = df

In [ ]:
df_Inicial.info()

## Detección y tratamiento de datos faltantes

### Analizamos los datos de 1 paciente con las muestras ordenadas cada 15 min

In [ ]:
pacienteID="LIB193277"
paciente = utils.getDataPatient(df,pacienteID,'15min')
paciente

Realizamos un análisis exploratorio

In [ ]:
profile = ProfileReport(paciente, title="Pandas Profiling Report",
    missing_diagrams=None,
    correlations=None, interactions=None,samples=None)
profile.to_file("analisisExploratorio/analisis_paciente_ID_LIB193277.html")

In [ ]:
paciente_Null=paciente.isnull()
paciente_Null.sum()

Se observa que no todos los registros fueron tomados cada 15 min, y existen datos faltantes

In [ ]:
paciente_Null.tail()

In [ ]:
paciente['Glucose level'].plot(figsize=(20, 10))

In [ ]:
import missingno as msno
msno.matrix(paciente)

### Estrategia de reemplazo de datos faltantes

In [ ]:
ObjRangeDateStart='2020-6-12';
ObjRangeDateEnd='2020-6-25';

In [ ]:
utils.plotRangeDates(paciente,ObjRangeDateStart,ObjRangeDateEnd)

Intentar usar -1 como valor por defecto para datos faltantes

In [ ]:
paciente_fill=utils.fillNullData(paciente,'-1')
utils.plotTwoDf(paciente_fill,paciente,ObjRangeDateStart,ObjRangeDateEnd)

ffill reemplaza los valores faltantes con el valor de la muestra anterior

In [ ]:
paciente_fill=utils.fillNullData(paciente,'ffill')
utils.plotTwoDf(paciente_fill,paciente,ObjRangeDateStart,ObjRangeDateEnd)

bfill reemplaza los valores faltantes con el valor de la muestra siguiente

In [ ]:
paciente_fill=utils.fillNullData(paciente,'bfill')
utils.plotTwoDf(paciente_fill,paciente,ObjRangeDateStart,ObjRangeDateEnd)

interpolate_linear reemplaza los valores faltantes con un valor que pasa por la linea entre 2 puntos datos

In [ ]:
paciente_fill=utils.fillNullData(paciente,'interpolate_linear')
utils.plotTwoDf(paciente_fill,paciente,ObjRangeDateStart,ObjRangeDateEnd)

In [ ]:
paciente_fill_Null=paciente_fill.isnull()
paciente_fill_Null.sum()

###	Normalización de los datos

En este paso solo vamos a entrenar los normalizadores que vamos a utilizar, luego integraremos todos los pasos en una función general

#### Entrenamiento para datos de glucosa

In [ ]:
df['Glucose level'].head()

In [ ]:
scalerGlucosa = MinMaxScaler(feature_range=(0,1))

df[['Glucose level']]=scalerGlucosa.fit_transform(df[['Glucose level']].values)
dump(scalerGlucosa, open('dataset/generado/scalerGlucosa.scaler', 'wb'))

In [ ]:
df

In [ ]:
df[['Glucose level']]=scalerGlucosa.inverse_transform(df[['Glucose level']].values)

In [ ]:
df

#### Entrenamiento para datos de horas

In [ ]:
arrayHours=[]
for i in range(24):
    arrayHours.append(i)


dfHours= pd.DataFrame(arrayHours)
dfHours.columns = ['hours']
dfHours

In [ ]:
scalerHours = MinMaxScaler()

arrayHours=scalerHours.fit_transform(dfHours[['hours']].values)
dump(scalerHours, open('dataset/generado/scalerHours.scaler', 'wb'))
arrayHours

In [ ]:
arrayHoursT=scalerHours.inverse_transform([[1]])
arrayHoursT[0]

#### Entrenamiento para datos de minutos

In [ ]:
arrayMins=[]
for i in range(60):
    arrayMins.append(i)


dfMins= pd.DataFrame(arrayMins)
dfMins.columns = ['mins']
dfMins

In [ ]:
scalerMin = MinMaxScaler()

arrayMins=scalerMin.fit_transform(dfMins[['mins']].values)
dump(scalerMin, open('dataset/generado/scalerMin.scaler', 'wb'))
arrayMins

In [ ]:
arrayMinT=scalerMin.inverse_transform([[1]])
arrayMinT[0]

#### Entrenamiento para datos de partes del día

In [ ]:
arrayPod=[]
for i in range(6):
    arrayPod.append(i+1)


dfPod= pd.DataFrame(arrayPod)
dfPod.columns = ['podid']
dfPod

In [ ]:
scalerPodId = MinMaxScaler()

arrayPod=scalerPodId.fit_transform(dfPod[['podid']].values)
dump(scalerPodId, open('dataset/generado/scalerPodId.scaler', 'wb'))
arrayPod

In [ ]:
arrayPodT=scalerPodId.inverse_transform([[0.6]])
arrayPodT[0]

#### Entrenamiento para datos de niveles de glucosa

In [ ]:
arrayLevel=[-1,0,1]
dfLevel= pd.DataFrame(arrayLevel)
dfLevel.columns = ['levelid']
dfLevel

In [ ]:
scalerLevelId = MinMaxScaler()

arrayLevel=scalerLevelId.fit_transform(dfLevel[['levelid']].values)
dump(scalerLevelId, open('dataset/generado/scalerLevelId.scaler', 'wb'))
arrayLevel

In [ ]:
arrayLevelT=scalerLevelId.inverse_transform([[0.5]])
arrayLevelT[0]

### Ingeniería de características

In [ ]:
df

In [ ]:
paciente_data = utils.getDataPatient(df,pacienteID,'15min',False)

In [ ]:
paciente_data

In [ ]:
paciente_data=utils.generateNewColumns(paciente_data,scalerLevelId,scalerHours,scalerMin,scalerPodId,scalerGlucosa,True)

In [ ]:
paciente_data